<a href="https://colab.research.google.com/github/martinpius/Behind-Keras-Layers-and-Models/blob/main/Customized_keras_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount("/content/drive/", force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f"You are on CoLab with tensorflow version: {tf.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n...please load your drive...")
  COLAB = False
def time_fmt(x:float = 123.4987)->float:
  h = int(x / (60 * 60))
  m = int(x % (60 * 60) / 60)
  s = int(x % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f"...time testing...time testing...time testing...\ntime elapse: <<<{time_fmt()}>>>")

Mounted at /content/drive/
You are on CoLab with tensorflow version: 2.4.1
...time testing...time testing...time testing...
time elapse: <<<0: 02: 03.00>>>


In [6]:
import numpy as np
import time, os
import tensorflow as tf

In [8]:
#Consider the following simple MLP with 3 layers using model subclassing:

In [38]:
#We first construct our layers from scratch like as follows:
class DenseLayer(tf.keras.layers.Layer):
  def __init__(self, units, inputs_dim, *args, **kwargs):
    super(DenseLayer, self).__init__(*args, **kwargs)
    self.w = self.add_weight(shape = (inputs_dim, units),
                             name = 'weights', initializer = 'random_normal',
                             trainable = True)
    self.b = self.add_weight(shape = (units,), initializer = 'zeros', name = 'bias',
                             trainable = True)
  
  def call(self, inputs_tensor):
    x = tf.matmul(inputs_tensor, self.w) + self.b
    return tf.nn.relu(x)


In [45]:
class SimpleMLP(tf.keras.models.Model):
  def __init__(self, num_classes = 10, *args, **kwargs):
    super(SimpleMLP, self).__init__(*args, **kwargs)
    self.dense1 = DenseLayer(units = 128, inputs_dim = 784, name = 'dense1')
    self.dense2 = DenseLayer(units = 64, inputs_dim = 128, name = 'dense2')
    self.outputs = DenseLayer(units = 10, inputs_dim = 64, name = 'outputs')

  def call(self, inputs_tensor, training = False):
    x = self.dense1(inputs_tensor)
    x = tf.nn.relu(x)
    x = self.dense2(x)
    x = tf.nn.relu(x)
    return tf.nn.softmax(self.outputs(x))

In [46]:
#We can simply instantiate compile and train this model as follows:

In [47]:
model = SimpleMLP(name = 'MLP_CUSTOMIZED')

In [28]:
#Load the mnist dataset:

In [29]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [30]:
print(f"x_train_shape: {x_train.shape}, x_test_shape: {x_test.shape}\ny_train_shape: {y_train.shape}, y_test_shape: {y_test.shape}")

x_train_shape: (60000, 28, 28), x_test_shape: (10000, 28, 28)
y_train_shape: (60000,), y_test_shape: (10000,)


In [31]:
x_train, x_test = x_train.reshape((60000, 784)).astype(np.float32)/255.0, x_test.reshape((10000, 784)).astype(np.float32)/255.0

In [32]:
y_train, y_test = tf.keras.utils.to_categorical(y_train, num_classes = 10), tf.keras.utils.to_categorical(y_test, num_classes = 10)

In [33]:
#Convert to tensorflow data type:

In [42]:
EPOCHS = 20
BATCH_SIZE = 64
BUFFER = len(x_train)
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(BUFFER).batch(batch_size = BATCH_SIZE, drop_remainder = True)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.batch(batch_size = BATCH_SIZE, drop_remainder = True)
x_sample_batch_train, y_sample_batch_train = next(iter(train_data))
print(f"x_batch_shape: {x_sample_batch_train.shape}, y_batch_shape: {y_sample_batch_train.shape}")

x_batch_shape: (64, 784), y_batch_shape: (64, 10)


In [43]:
#training loop from scratch:

In [48]:
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001)
loss_obj = tf.keras.losses.CategoricalCrossentropy(from_logits = False)
train_metric = tf.keras.metrics.CategoricalAccuracy()
eval_metric = tf.keras.metrics.CategoricalAccuracy()
tic = time.time()
for epoch in range(EPOCHS):
  print(f"...training start for epoch: {epoch + 1}....\n....please wait....it may take a while.... ")
  for (step, (x_train_batch, y_train_batch)) in enumerate(train_data):
    with tf.GradientTape() as tape:
      preds = model(x_train_batch, training = True)
      train_loss = loss_obj(y_train_batch, preds)
    grads = tape.gradient(train_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_metric.update_state(y_train_batch,preds)
    train_acc = train_metric.result()
    train_metric.reset_states()
    if step % 200 == 0:
      print(f"epoch: {epoch + 1}, train accuracy: {float(train_acc):.4f}")
      print(f"batch number: {step}, train loss: {float(train_loss):.4f}")
  for (step, (x_val_batch, y_val_batch)) in enumerate(test_data):
    preds = model(x_val_batch, training = False)
    val_loss = loss_obj(y_val_batch, preds)
    eval_metric.update_state(y_val_batch, preds)
    val_acc = eval_metric.result()
    eval_metric.reset_states()
    if step % 200 == 0:
      print(f"epoch: {epoch + 1}, validation accuracy: {float(val_acc):.4f}")
      print(f"batch number: {step}, validation loss: {float(val_loss):.4f}")
toc = time.time()
print(f"\ntotal training and evaluation time for this model is: {time_fmt(toc - tic)}")


...training start for epoch: 1....
....please wait....it may take a while.... 
epoch: 1, train accuracy: 0.0781
batch number: 0, train loss: 2.3047
epoch: 1, train accuracy: 0.8750
batch number: 200, train loss: 0.4781
epoch: 1, train accuracy: 0.7500
batch number: 400, train loss: 0.7242
epoch: 1, train accuracy: 0.8594
batch number: 600, train loss: 0.4173
epoch: 1, train accuracy: 0.8125
batch number: 800, train loss: 0.5902
epoch: 1, validation accuracy: 0.9531
batch number: 0, validation loss: 0.1427
...training start for epoch: 2....
....please wait....it may take a while.... 
epoch: 2, train accuracy: 0.8906
batch number: 0, train loss: 0.3303
epoch: 2, train accuracy: 0.9062
batch number: 200, train loss: 0.2795
epoch: 2, train accuracy: 0.9375
batch number: 400, train loss: 0.2002
epoch: 2, train accuracy: 0.8281
batch number: 600, train loss: 0.4485
epoch: 2, train accuracy: 0.8906
batch number: 800, train loss: 0.3394
epoch: 2, validation accuracy: 0.9688
batch number: 0, va